
# 1 - Sequence to Sequence Learning with Neural Networks
In this series we'll be building a machine learning model to go from once sequence to another, using PyTorch and torchtext. This will be done on German to English translations, but the models can be applied to any problem that involves going from one sequence to another, such as summarization, i.e. going from a sequence to a shorter sequence in the same language.

In this first notebook, we'll start simple to understand the general concepts by implementing the model from the Sequence to Sequence Learning with Neural Networks paper.

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torchtext.datasets import Multi30k
from torchtext.data import Field,BucketIterator

import spacy
import numpy as np 
import random
import math
import time

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [7]:
SRC = Field(tokenize=tokenize_de,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRC = Field(tokenize=tokenize_en,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)


In [8]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de','.en'),
                                                   fields=(SRC,TRC))

downloading training.tar.gz


.data\multi30k\training.tar.gz: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.21M/1.21M [00:02<00:00, 592kB/s]


downloading validation.tar.gz


.data\multi30k\validation.tar.gz: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46.3k/46.3k [00:00<00:00, 206kB/s]


downloading mmt_task1_test2016.tar.gz


.data\multi30k\mmt_task1_test2016.tar.gz: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66.2k/66.2k [00:00<00:00, 180kB/s]


In [9]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [11]:
vars(train_data.examples[0])

{'src': ['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'im',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 'trg': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.']}

In [12]:
SRC.build_vocab(train_data,min_freq=2)
TRC.build_vocab(train_data,min_freq=2)

In [14]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRC.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


In [15]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
BATCH_SIZE=8
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data,test_data),
    batch_size=BATCH_SIZE,
    device=device
    )


In [23]:
class Encoder(nn.Module):
    def __init__(self,input_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        
        self.embedding= nn.Embedding(input_dim,emb_dim)
        
        self.rnn = nn.LSTM(emb_dim,hid_dim,n_layers,dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
    def forward(self,src):
        
        embedd = self.dropout(self.embedding(src))
        
        outputs,(hidden,cell) = self.rnn(embedd)
        
        return hidden,cell
        

In [65]:
class Decoder(nn.Module):
    def __init__(self,output_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.output_dim=output_dim
        self.hid_dim=hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim,emb_dim)
        
        self.rnn  = nn.LSTM(emb_dim,hid_dim,n_layers,dropout=dropout)
        
        self.fc_out= nn.Linear(hid_dim,output_dim)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,input,hidden,cell):
        input= input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        outputs,(hidden,cell) = self.rnn(embedded)
        prediction = self.fc_out(outputs.squeeze(0))
        return prediction,hidden,cell
    
        
        

In [66]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device= device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
    
    def forward(self,src,trg , teacher_forcing_ratio=0.5):
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size=self.decoder.output_dim
        
        outputs = torch.zeros(trg_len,batch_size,trg_vocab_size).to(self.device)
        hidden,cell = self.encoder(src)
        
        input = trg[0,:]
        
        for t in range(1,trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t]= output
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        return outputs

In [67]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRC.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [68]:

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [69]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [70]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


In [71]:
optimizer = optim.Adam(model.parameters())

In [72]:
TRG_PAD_IDX= TRC.vocab.stoi[TRC.pad_token]

In [73]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [74]:
def train(model,iterator,optimizer,criterion , clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src=batch.src
        trg=batch.trg
        
        optimizer.zero_grad()
        
        output = model(src,trg)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

        

In [75]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [76]:

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

NameError: name 'epoch_time' is not defined